<a href="https://colab.research.google.com/github/hanyang-lee/R_PY_Computing/blob/master/vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import

In [1]:
!nvidia-smi

Sun Jun 12 23:49:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.models as models

from torchsummary import summary

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Hyperparameter 설정

In [4]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':0.05,
    'SEED':41,
}

seed 고정

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

Define Model Architecture

In [6]:
class VGG(torch.nn.Module):
  def __init__(self, cifar=False):
    super(VGG, self).__init__()
    self.layer1 = torch.nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer2 = torch.nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
        
    self.layer3 = torch.nn.Sequential(
        nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer4 = torch.nn.Sequential(
        nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer5 = torch.nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.adapt = nn.AdaptiveAvgPool2d((1,1)) #이건 왜 있는걸까? (batch,1,1,512)

    self.classifier= nn.Sequential(
      nn.Linear(512,512),
      nn.ReLU(),
      nn.Dropout(0.5),
      nn.Linear(512,512),
      nn.ReLU(),
      nn.Dropout(0.5),
      nn.Linear(512,1000)
      )
      
    if cifar:
      self.classifier = nn.Sequential(
          nn.Linear(512,512),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(512,258),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(258,10)
      )

  def forward(self, x):
    #cnn model (batch, 3, 224, 224) -> (batch, 512, 7, 7)
    #(batch, 3, 224, 224)
    x = self.layer1(x)
    #(batch, 64, 112, 112)
    x = self.layer2(x)
    #(batch, 128, 56, 56)
    x = self.layer3(x)
    #(batch, 256, 28, 28)
    x = self.layer4(x)
    #(batch, 512, 14, 14)
    x = self.layer5(x)
    x = self.adapt(x)
    #(batch, 512, 7, 7) -> Flatten(batch, 512*7*7(=25088))
    x= torch.flatten(x, start_dim=1)
    #Regressor (batch, 25088) -> (batch, 1000)
    out = self.classifier(x)
    return out

train, test data setting

In [7]:
transform_train = transforms.Compose([
    # transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=0,std=1)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=0,std=1),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


make model

In [8]:
model = VGG(cifar=True)
model = model.to(device)

In [9]:
model_name = 'model.pth'

loss_fn = nn.CrossEntropyLoss() #softmax의 log 결과를 cross entropy loss값의 결과로 내주는 함수
optimizer = optim.SGD(model.parameters(), lr=CFG["LEARNING_RATE"], momentum=0.9, weight_decay=0.0001)

train_loss = 0
valid_loss = 0
correct = 0
total_cnt = 0
best_acc = 0

train and test

In [10]:
epoch_num = CFG["EPOCHS"]
for epoch in range(epoch_num):
  print(f"====== { epoch+1} epoch of { epoch_num } ======")
  model.train()
  
  train_loss = 0
  test_loss = 0
  correct = 0
  total_cnt = 0

  for step, (x,y) in enumerate(train_loader):
    x ,y = x.to(device), y.to(device)
    
    optimizer.zero_grad() #gradient를 zero로

    logits = model(x) # (batch, # class)
    loss = loss_fn(logits, y) 
    loss.backward()

    optimizer.step()
    train_loss += loss.item()
    predict = logits.argmax(dim=1) # (batch,)

    total_cnt += y.size(0)
    correct += predict.eq(y).sum().item() # (predict==y).sum().item()

    if step % 100 == 0 and step != 0:
      print(f"\n====== { step } Step of { len(train_loader) } ======")
      print(f"Train Acc : { correct / total_cnt }")
      print(f"Train Loss : { loss.item() / y.size(0) }")
  
  correct = 0
  total_cnt = 0

  with torch.no_grad(): #autograd engine을 비활성화
    model.eval() #해당 모델의 모든 layer를 evaluation mode로 torch.no_grad()와 eval() 함수는 같이 쓰임
    for step, batch in enumerate(test_loader):
      batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
      total_cnt += batch[1].size(0)
      logits = model(batch[0])
      test_loss += loss_fn(logits, batch[1])
      _, predict = logits.max(1)
      correct += predict.eq(batch[1]).sum().item()
    test_acc = correct / total_cnt
    print(f"\nTest Acc : { test_acc }")    
    print(f"Test Loss : { test_loss / total_cnt }")

    if(test_acc > best_acc):
      best_acc = test_acc
      torch.save(model, model_name)
      print("Model Saved!")

====== 1 epoch of 10 ======

====== 100 Step of 391 ======
Train Acc : 0.0999381188118812
Train Loss : 0.01802205480635166

====== 200 Step of 391 ======
Train Acc : 0.09771455223880597
Train Loss : 0.017991380766034126

====== 300 Step of 391 ======
Train Acc : 0.09946013289036545
Train Loss : 0.018049923703074455

Test Acc : 0.1
Test Loss : 0.018199510872364044
Model Saved!
====== 2 epoch of 10 ======

====== 100 Step of 391 ======
Train Acc : 0.0968440594059406
Train Loss : 0.017996691167354584

====== 200 Step of 391 ======
Train Acc : 0.09678171641791045
Train Loss : 0.017963483929634094

====== 300 Step of 391 ======
Train Acc : 0.09665697674418605
Train Loss : 0.017977874726057053

Test Acc : 0.1
Test Loss : 0.01820354349911213
====== 3 epoch of 10 ======

====== 100 Step of 391 ======
Train Acc : 0.1021039603960396
Train Loss : 0.017952146008610725

====== 200 Step of 391 ======
Train Acc : 0.10366138059701492
Train Loss : 0.01799849607050419

====== 300 Step of 391 ======
Trai